# Семинар 1: Разведочный анализ данных.

Решение любой задачи, связанной с машинным обучением, начинается с разведочного анализа данных. Перед тем, как строить модели, надо понять, сколько у нас данных и какая информация есть о каждом объекте, а также:
* выяснить, нет ли пропусков в данных (т.е. отсутствующих значений признаков у некоторых объектов);
* выяснить, нет ли выбросов (т.е. объектов, которые очень сильно отличаются от большинства, имеют неадекватные значения признаков);
* выяснить, нет ли повторов в данных;
* выяснить, нет ли нелогичной информации (например, если мы анализируем данные по кредитам, и видим запись, где кредит выдали пятилетнему ребёнку, то это странно).

И это лишь небольшой список проблем, которые можно выявить. Помимо этого с данными нужно в целом познакомиться, чтобы понять, какие признаки там можно сделать, какие из них будут наиболее полезны.

## Данные:

В машинном обучении данные удобно представлять в виде таблиц. Каждая строка отвечает объекту, а каждый столбец - признаку. Работа с таблицами в питоне реализована в библиотеке `pandas`.

https://pandas.pydata.org/docs/

In [ ]:
import numpy as np
import pandas as pd

Основной объект в `pandas` - это `DataFrame`. Его можно создать напрямую или прочитать из файла.

In [ ]:
df = pd.DataFrame({
    'AAA': [4, 5, 6, 7],
    'BBB': [10, 20, 30, 40],
    'CCC': [100, 50, 'E', -50]
})

In [ ]:
type(df)

In [ ]:
df

Данные можно скачать здесь: https://www.kaggle.com/c/titanic/data?select=train.csv .

In [ ]:
# Либо можно скачать так:
!wget https://raw.githubusercontent.com/SergeyKorpachev/math-faculty-ml/refs/heads/main/2026/seminars/seminar01/train.csv

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df

Mетоды `.info()` и `.describe()` позволяют посмотреть на описание колонок и статистики по численным колонкам.

In [ ]:
df.info()

In [ ]:
df.describe()

Также в `pandas` есть тип `Series`, который отвечает за строки и столбцы. Например, колонка в таблице - это `Series`.

In [ ]:
df["Name"] # или можно ещё df.Name

Можно оставить несколько колонок.

In [ ]:
df[["Name", "Age", "Ticket"]]

А ещё `Series` ведёт себя как numpy массив и можно делать вот так:

In [ ]:
df[(df["Age"] > 20) & (df["Pclass"] == 2)]

Это также можно сделать (IMHO удобнее) через `.query`.

In [ ]:
df.query("Age > 20 & Pclass == 2")

Для индексации по таблице также существуют два способа - через `.loc` и `.iloc`. Например:

In [ ]:
df.loc[4]

или можно получить один элемент

In [ ]:
df.loc[4, "Age"]

можно делать срезы таблицы

In [ ]:
df.loc[4:10, "Age"]

In [ ]:
df.loc[4:10, "Pclass":"Age"]

`.iloc` далает то же самое, только не по названию колонок и индексу, а по номерам строк и столбцов.

In [ ]:
samples = df.sample(10)
samples

In [ ]:
samples.loc[samples.index[3]]

In [ ]:
samples.iloc[2] # если тут сделать для samples.index[3], то такого индекса может не быть

In [ ]:
df.iloc[4:10, 2:5]

Также в таблицу можно добавлять новые колонки.

In [ ]:
import random

df["New column"] = df["Age"] ** ((3 - df["Pclass"]) / 5)
df["Surname"] = df["Name"].str.split(",").str[0]
df["Yet another column"] = df.apply(lambda row: random.random() * (row["Age"] + row["Pclass"]), axis=1)

In [ ]:
df

В данных бывают пропуски. Это важно учитывать. Заполнять пропуски необходимо в соответствии со смыслом колонки. Можно заполнять с помощью среднего, медианного, константного или других значений. Для этого обычно используется метод `.fillna()`.

In [ ]:
df.isna().sum()

Также есть полезный метод `.groupby()`, который группируюет объекты таблици по значениям колонки. Например:

In [ ]:
df.groupby("Sex")["Fare"].mean()

In [ ]:
df.groupby(["Sex", "Pclass"])["Fare"].mean()

## Визуализация

Но куда приятнее изучать данные с помощью картинок. Для такого тоже есть библиотеки, например, `seaborn`.

Полезные ссылки:
- https://www.python-graph-gallery.com/
- https://seaborn.pydata.org/examples/index.html

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.histplot(data=df, x="Age")

In [ ]:
sns.relplot(data=df, x="Age", y="Fare", hue="Survived")

In [ ]:
sns.violinplot(data=df, x="Embarked", y="Age", hue="Survived", split=True)

In [ ]:
cols = ["Age", "Fare", "New column", "Yet another column", "Survived"]

sns.pairplot(data=df[cols], hue="Survived") # это может занять какое-то время

In [ ]:
# Упражнение: выживаемость по классу.
# Постройте столбчатую диаграмму (countplot) выживыемости пассажира относительно его класса.
# Сделайте подписи (title, xlabel, ylabel) к осям и к самой диаграмме.

# Ваш код сюда :)

In [ ]:
from sklearn.datasets import fetch_california_housing

california_housing = fetch_california_housing(as_frame=True)
california_housing_df = california_housing.frame
california_housing_df

In [ ]:
print(california_housing.DESCR)

In [ ]:
sns.heatmap(california_housing_df.corr())

In [ ]:
sns.scatterplot(data=california_housing_df, x="Longitude", y="Latitude",
                size="MedHouseVal", hue="MedHouseVal",
                palette="viridis", alpha=0.5)

In [ ]:
bins = pd.qcut(california_housing_df["MedHouseVal"], 6, retbins=False)

california_housing_df["MedHouseValBin"] = bins.apply(lambda x: x.mid)

In [ ]:
sns.histplot(data=california_housing_df, x="MedInc", hue="MedHouseValBin", multiple="stack")

In [ ]:
sns.scatterplot(data=california_housing_df, x="AveRooms", y="AveOccup", hue="MedHouseVal")

In [ ]:
california_housing_filt = california_housing_df.query("AveOccup < AveOccup.quantile(0.999) & AveRooms < AveRooms.quantile(0.999)")
sns.scatterplot(california_housing_filt, x="AveRooms", y="AveOccup",
                hue="MedHouseVal", alpha=0.5, palette="viridis")

Давайте посмотрим на население.

In [ ]:
sns.histplot(data=california_housing_df["Population"], bins=100)

In [ ]:
sns.histplot(data=np.log1p(california_housing_df["Population"]), bins=100)

## DIY: Japan credit screening data

https://archive.ics.uci.edu/ml/datasets/Japanese+Credit+Screening

Помогите Даше изучить данные японского кредитного скрининга.

<img src="https://www.meme-arsenal.com/memes/e9e029d4bb638ba6156361ffbe3e41f8.jpg"
     width="400"
     height="auto">

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data

In [ ]:
column_names = [f"A{i}" for i in range(1, 17)]
data = pd.read_csv("./crx.data", names=column_names, na_values="?")
data

In [ ]:
# Проанализируйте это досконально!